In [1]:
import pandas as pd
import numpy as np
import re
import time
import ast

from os import path

from gensim.models import Word2Vec, KeyedVectors
from tqdm import tqdm

tqdm.pandas()

data_dir = path.join('..', 'data')
img_dir = path.join(data_dir, 'img')
temp_dir = path.join(data_dir, 'temp')

dataset_file = 'merged_data4_with_sentiment.csv'

data = pd.read_csv(path.join(data_dir, dataset_file), sep='`')
data = data.sort_values(by=['timestamp'])

c:\users\techw\appdata\local\programs\python\python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
c:\users\techw\appdata\local\programs\python\python38\lib\site-packages\tqdm\std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
def hstack_title_body_tokens(title, body):
    tokens = list(title)
    filtered_body_tokens = [body_tokens for body_tokens in body if body_tokens != '[\'nan\']']

    for list_ in filtered_body_tokens:
        tokens.append(list_)

    tokens = [ast.literal_eval(res) for res in tokens]
    assert len(tokens) == len(filtered_body_tokens) + len(data.title_tokens.values)
    
    return tokens

In [4]:
# tokens = list(data.title_tokens.values)
# filtered_body_tokens = [body_tokens for body_tokens in data.body_tokens.values if body_tokens != '[\'nan\']']

# for list_ in filtered_body_tokens:
#     tokens.append(list_)
    
# tokens = [ast.literal_eval(res) for res in tokens]
# assert len(tokens) == len(filtered_body_tokens) + len(data.title_tokens.values)

stem_tokens = hstack_title_body_tokens(data.title_tokens, data.body_tokens)

In [16]:
def fit(tokens, vector_size=100):
    start_time = time.time()
    model = Word2Vec(
                sentences=tokens, 
                sg=1, 
                vector_size=300,  
                workers=4
            )

    print(f'Time taken : {(time.time() - start_time) / 60:.2f} mins')
    return model

In [17]:
def save(model, name):
    model.wv.save_word2vec_format(path.join(data_dir, name))

# save(model, 'wsb_embedding_300d.txt')
# How to load:
# w2v = KeyedVectors.load_word2vec_format(path.join(data_dir, 'wsb_embedding_300d.txt'))

# How to get vector using loaded model
# w2v.get_vector('gme')

# ----------------------------------------------------

In [5]:
temp = pd.read_csv(path.join(data_dir, 'data_temp.csv'), sep='`')

In [10]:
data['title_stem_tokens'] = temp.title_stem_tokens
data['body_stem_tokens'] = temp.body_stem_tokens
data['title_tokens'] = temp.title_tokens
data['body_tokens'] = temp.body_tokens

In [11]:
tokens = hstack_title_body_tokens(data.title_tokens, data.body_tokens)

In [18]:
model = fit(tokens)
save(model, 'wsb_embedding_100d.txt')

stem_model = fit(stem_tokens)
save(stem_model, 'wsb_stem_embedding_100d.txt')

Time taken : 0.29 mins
Time taken : 0.22 mins
